In [165]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [166]:
class gaussian_lstm(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers, batch_size):
        super(gaussian_lstm, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.embed = nn.Linear(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True)#nn.ModuleList([nn.LSTMCell(hidden_size, hidden_size) for i in range(self.n_layers)])
        self.mu_net = nn.Linear(hidden_size, output_size)
        self.logvar_net = nn.Linear(hidden_size, output_size)
        # self.hidden = self.init_hidden()

    # def init_hidden(self, batch_size=1):
    #     hidden = []
    #     for i in range(self.n_layers):
    #         hidden.append((Variable(torch.zeros(batch_size, self.hidden_size)),
    #                        Variable(torch.zeros(batch_size, self.hidden_size))))
    #     self.hidden = hidden
    #     return hidden

    # def init_hidden_(self):
    #     hidden = []
    #     for i in range(self.n_layers):
    #         hidden.append((Variable(torch.zeros(self.batch_size, self.hidden_size)),
    #                        Variable(torch.zeros(self.batch_size, self.hidden_size))))
    #     self.hidden = hidden
        #return hidden

    def reparameterize(self, mu, logvar):
        logvar = logvar.mul(0.5).exp_()
        eps = Variable(logvar.data.new(logvar.size()).normal_())
        #return eps.add_(mu)
        #return eps.mul(logvar)
        return eps.mul(logvar).add_(mu)

    def forward(self, input):
        #import pdb
        #pdb.set_trace()
        # embedded = self.embed(input.view(-1, self.input_size))
        # h_in = embedded
        # for i in range(self.n_layers):
        #     # self.hidden[i] = self.lstm(h_in, self.hidden[i])
        #     h_in = nn.Linear(3, 3)(h_in)#self.hidden[i][0]
        # mu = self.mu_net(h_in)
        # logvar = self.logvar_net(h_in)
        # z = self.reparameterize(mu, logvar)
        # return z, mu, logvar
        return self.embed(input)

In [167]:
torch_model = gaussian_lstm(input_size=5, output_size=2, hidden_size=3, n_layers=1, batch_size=1)

In [168]:
import random
import numpy as np
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch_model(torch.tensor([[1, 1, 1, 1, 1]], dtype=torch.float32))

tensor([[-0.6163,  0.7211, -0.8625]], grad_fn=<AddmmBackward0>)

In [169]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, LSTM, Input
import tensorflow_probability as tfp

In [170]:
input = Input(shape=(5))
# x = LSTM(3)(input)
x = Dense(3)(input)
# mu = Dense(2)(x)
# logvar = Dense(2)(x)
tf_model = Model(inputs=input, outputs=[x])

In [182]:
weights = torch_model.embed.weight.detach().numpy().T
weights

array([[-0.00334821,  0.11992361, -0.1351538 ],
       [ 0.23990487, -0.00886072, -0.08790671],
       [-0.36807698,  0.35459095, -0.42724484],
       [-0.32912195, -0.03968754, -0.29618156],
       [-0.17224626,  0.11833833, -0.18435177]], dtype=float32)

In [188]:
bias = torch_model.embed.bias.detach().numpy()
bias

array([0.01656639, 0.17679927, 0.26833835], dtype=float32)

In [189]:
tf_model.set_weights([weights, bias])

In [190]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
tf_model(tf.constant([[1, 1, 1, 1, 1]], dtype=tf.float32))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.6164197 ,  0.72104216, -0.862399  ]], dtype=float32)>

In [173]:
tf_model.get_weights()

[array([[-0.3603096 , -0.5082418 ,  0.06129855],
        [ 0.1061011 , -0.144324  ,  0.5331736 ],
        [-0.01173449,  0.86278516,  0.34075528],
        [-0.64887196,  0.36341316,  0.28131205],
        [ 0.12515205, -0.23425382, -0.1376664 ]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [177]:
torch_model.embed.weight.detach().numpy()

array([[-0.00334821,  0.23990487, -0.36807698, -0.32912195, -0.17224626],
       [ 0.11992361, -0.00886072,  0.35459095, -0.03968754,  0.11833833],
       [-0.1351538 , -0.08790671, -0.42724484, -0.29618156, -0.18435177]],
      dtype=float32)

In [178]:
torch_model.embed.bias

Parameter containing:
tensor([0.0166, 0.1768, 0.2683], requires_grad=True)